In [1]:
#Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import re
from collections import defaultdict

In [2]:
#Use one ./ if running from run_notebooks.py
#Use two ../if running directly from Jupyter Notebooks
#file_beg = '../NHANES-Downloader/data/csv_data/'

file_beg = './NHANES-Downloader/data/csv_data/'

In [3]:
#Import all the files
files1 = glob.glob(file_beg+'1999-2000/*/*.csv')
files2 = glob.glob(file_beg+'2001-2002/*/*.csv')
files3 = glob.glob(file_beg+'2003-2004/*/*.csv')
files4 = glob.glob(file_beg+'2005-2006/*/*.csv')
files5 = glob.glob(file_beg+'2007-2008/*/*.csv')
files6 = glob.glob(file_beg+'2009-2010/*/*.csv')
files7 = glob.glob(file_beg+'2011-2012/*/*.csv')
files8 = glob.glob(file_beg+'2013-2014/*/*.csv')
files9 = glob.glob(file_beg+'2015-2016/*/*.csv')

In [4]:
#Add files into a list
file_list = [files1, files2, files3, files4, files5, 
          files6, files7, files8, files9]

In [5]:
#Add files into a list of sorted and dictionaries
file_list_dict = []
for x in file_list:
    x.sort()
    file_list_dict.append(dict(enumerate(x)))

In [6]:
tchol_indx = [39, 44, 56, 74, 70, 73, 67, 85, 42]
hdl_indx = [46, 44, 44, 40, 54, 23]
trigly_indx = [40, 42, 55, 76, 73, 78, 70, 88, 44]

In [7]:
dfs_tc = defaultdict(int)
dfs_hdl = defaultdict(int)
dfs_tr = defaultdict(int)
for i in range(0,len(tchol_indx)):
    dfs_tc[i] = pd.read_csv(file_list_dict[i][tchol_indx[i]])
for i, x in enumerate(range(3,len(hdl_indx)+3)):
    dfs_hdl[i] = pd.read_csv(file_list_dict[x][hdl_indx[i]])
for i in range(0,len(trigly_indx)):
    dfs_tr[i] = pd.read_csv(file_list_dict[i][trigly_indx[i]])

## Join Columns Together

In [8]:
def innerjoin_df(dfs_list):
    df_join = dfs_list[0]
    for d in dfs_list[1:]:
        df_join = df_join.merge(d, how='inner', on=['SEQN'])
    return df_join

In [9]:
f = innerjoin_df([dfs_tc[3], dfs_tr[3]])
s = innerjoin_df([f, dfs_hdl[0]])

In [10]:
dfs = defaultdict(int)
for i in range(0,len(dfs_tc)):
    j = innerjoin_df([dfs_tc[i], dfs_tr[i]])
    if (i>2):
        j = innerjoin_df([dfs_hdl[i-3],j])
    dfs[i] = j

## Important Variables

In [11]:
#1999-2002
var_names = ['SEQN', 'LBXTC', 'LBDTCSI', 'LBDHDL', 'LBDHDLSI', 'LBXTR', 'LBDTRSI', 'LBDLDL', 'LBDLDLSI']

#2003-2004
var_names1 = ['SEQN', 'LBXTC', 'LBDTCSI', 'LBXHDD', 'LBDHDDSI', 'LBXTR', 'LBDTRSI', 'LBDLDL', 'LBDLDLSI']

#2005-2016
var_names2 = ['SEQN', 'LBXTC', 'LBDTCSI', 'LBDHDD', 'LBDHDDSI', 'LBXTR', 'LBDTRSI', 'LBDLDL', 'LBDLDLSI']

In [12]:
#To display all columns in Jupyter Notebooks
pd.set_option('display.max_columns', 500)

## Functions: Recategorize values, Count Values, Drop Rows

In [13]:
#Recategorize function
def recategorize(df, name, replace_dict):
    df[name].replace(
    to_replace=replace_dict,
    inplace=True
)
    
#Count values function
def count_vals(df, name):
    df_count = df.groupby(name)['SEQN'].nunique()
    print(df_count,"\n\n","NaN: ", df[name].isnull().sum())
    
#Drop rows that include certain values
def drop_rows(df, name, val_list):
    df.drop(df[df[name].isin(val_list)].index, inplace=True)

## Make a copy of the dataframes

In [14]:
df0 = dfs[0][var_names].copy() #1999-2000
df1 = dfs[1][var_names].copy() #2001-2002
df2 = dfs[2][var_names1].copy() #2003-2004
df3 = dfs[3][var_names2].copy() #2005-2006
df4 = dfs[4][var_names2].copy() #2007-2008
df5 = dfs[5][var_names2].copy() #2009-2010
df6 = dfs[6][var_names2].copy() #2011-2012
df7 = dfs[7][var_names2].copy() #2013-2014
df8 = dfs[8][var_names2].copy() #2015-2016

## Recategorize values

In [15]:
#Nothing to recategorize

## Rename columns 1999 - 2016

In [16]:
#New column names
col_names = var_names

In [17]:
df0.columns = col_names
df1.columns = col_names
df2.columns = col_names
df3.columns = col_names
df4.columns = col_names
df5.columns = col_names
df6.columns = col_names
df7.columns = col_names
df8.columns = col_names

## Append years 1999 - 2016

In [18]:
years = ["1999-2000","2001-2002","2003-2004","2005-2006","2007-2008", 
        "2009-2010","2011-2012","2013-2014","2015-2016"]

In [19]:
frames = [df0, df1, df2, df3, df4, df5, df6, df7, df8]

In [20]:
#Add years as a column
for i, df in enumerate(frames):
    df["Year"] = years[i]

In [21]:
result_1999_2016 = pd.concat(frames, keys = years)

In [22]:
result_1999_2016_cleaned = result_1999_2016.copy()

In [23]:
len(result_1999_2016)

32420

## Count values for each column

In [24]:
len(col_names)

9

In [25]:
len(result_1999_2016)

32420

In [26]:
result_1999_2016_cleaned[:5]

SEQN  LBXTC  LBDTCSI  LBDHDL  LBDHDLSI  LBXTR  LBDTRSI  LBDLDL  \
1999-2000 0   2.0  215.0     5.56    54.0      1.39  128.0     1.45   136.0   
          1   3.0  129.0     3.34    30.0      0.78  202.0     2.28    58.0   
          2   5.0  279.0     7.21    42.0      1.08  347.0     3.92   168.0   
          3   7.0  245.0     6.34   105.0      2.73   62.0     0.70   127.0   
          4   8.0  162.0     4.19    67.0      1.74   33.0     0.37    88.0   

             LBDLDLSI       Year  
1999-2000 0      3.52  1999-2000  
          1      1.50  1999-2000  
          2      4.34  1999-2000  
          3      3.28  1999-2000  
          4      2.28  1999-2000

## Remove rows with missing values:

In [27]:
for i in range (1, len(col_names)):
    drop_rows(result_1999_2016_cleaned, col_names[i], [np.nan])

In [28]:
len(result_1999_2016_cleaned)

28547

In [29]:
result_1999_2016_cleaned.isnull().values.any()

False

## See percentage of removed values

In [30]:
before = len(result_1999_2016)
before

32420

In [31]:
after = len(result_1999_2016_cleaned)
after

28547

In [32]:
(before-after)/after

0.13567099870389182

In [33]:
result_1999_2016_cleaned

SEQN  LBXTC  LBDTCSI  LBDHDL  LBDHDLSI  LBXTR  LBDTRSI  \
1999-2000 0         2.0  215.0     5.56    54.0      1.39  128.0    1.450   
          1         3.0  129.0     3.34    30.0      0.78  202.0    2.280   
          2         5.0  279.0     7.21    42.0      1.08  347.0    3.920   
          3         7.0  245.0     6.34   105.0      2.73   62.0    0.700   
          4         8.0  162.0     4.19    67.0      1.74   33.0    0.370   
          5         9.0  148.0     3.83    58.0      1.49   56.0    0.630   
          6        10.0  140.0     3.62    51.0      1.31   45.0    0.510   
          7        11.0  132.0     3.41    40.0      1.02   76.0    0.860   
          8        12.0  156.0     4.03    38.0      0.98  146.0    1.650   
          9        15.0  199.0     5.15    58.0      1.49   54.0    0.610   
          10       20.0  145.0     3.75    43.0      1.10  117.0    1.320   
          11       21.0  161.0     4.16    34.0      0.89  120.0    1.350   
          12       23.0  160.0     4.14    32.0      0.83  171.0    1.930   
          13       25.0  159.0     4.11    55.0      1.42  202.0    2.280   
          15       28.0  151.0     3.90    64.0      1.65   52.0    0.590   
          16       29.0  216.0     5.59    49.0      1.28  167.0    1.890   
          17       31.0  150.0     3.88    40.0      1.04  128.0    1.450   
          19       34.0  195.0     5.04    41.0      1.06   97.0    1.100   
          21       42.0  187.0     4.84    47.0      1.22   62.0    0.700   
          23       44.0  108.0     2.79    35.0      0.91   47.0    0.530   
          24       46.0  187.0     4.84    36.0      0.93  122.0    1.380   
          25       49.0  176.0     4.55    42.0      1.09   86.0    0.970   
          26       53.0  139.0     3.59    50.0      1.29   71.0    0.800   
          27       55.0  285.0     7.37    66.0      1.70  210.0    2.370   
          28       57.0  243.0     6.28    46.0      1.20   97.0    1.100   
          29       58.0  172.0     4.45    68.0      1.76  104.0    1.170   
          30       59.0  266.0     6.88    58.0      1.50  109.0    1.230   
          33       62.0  168.0     4.34    70.0      1.82   75.0    0.850   
          34       64.0  195.0     5.04    41.0      1.07  155.0    1.750   
          35       67.0  149.0     3.85    47.0      1.21   94.0    1.060   
...                 ...    ...      ...     ...       ...    ...      ...   
2015-2016 3156  93572.0  139.0     3.59    38.0      0.98  217.0    2.450   
          3157  93573.0  197.0     5.09    69.0      1.78   91.0    1.027   
          3158  93577.0  155.0     4.01    63.0      1.63   39.0    0.440   
          3159  93582.0  133.0     3.44    63.0      1.63   40.0    0.452   
          3160  93589.0  222.0     5.74    67.0      1.73   43.0    0.485   
          3161  93590.0  189.0     4.89    55.0      1.42   60.0    0.677   
          3163  93592.0  233.0     6.03    57.0      1.47  123.0    1.389   
          3164  93593.0  213.0     5.51    46.0      1.19  114.0    1.287   
          3165  93595.0  164.0     4.24    74.0      1.91   40.0    0.452   
          3167  93600.0  170.0     4.40    87.0      2.25   42.0    0.474   
          3168  93602.0  145.0     3.75    59.0      1.53   58.0    0.655   
          3169  93607.0  200.0     5.17    33.0      0.85  236.0    2.664   
          3170  93608.0  201.0     5.20    70.0      1.81   73.0    0.824   
          3171  93611.0  169.0     4.37    49.0      1.27  134.0    1.513   
          3172  93612.0  184.0     4.76    45.0      1.16  378.0    4.268   
          3173  93619.0  205.0     5.30    74.0      1.91   60.0    0.677   
          3174  93624.0  261.0     6.75    61.0      1.58   47.0    0.531   
          3176  93639.0  140.0     3.62    54.0      1.40   47.0    0.531   
          3177  93643.0  245.0     6.34    63.0      1.63   79.0    0.892   
          3178  93645.0  138.0     3.57    52.0      1.34  130.0    1.468   
          3179 

## Remove Duplicated Measurements

In [34]:
result_1999_2016_cleaned.head()

SEQN  LBXTC  LBDTCSI  LBDHDL  LBDHDLSI  LBXTR  LBDTRSI  LBDLDL  \
1999-2000 0   2.0  215.0     5.56    54.0      1.39  128.0     1.45   136.0   
          1   3.0  129.0     3.34    30.0      0.78  202.0     2.28    58.0   
          2   5.0  279.0     7.21    42.0      1.08  347.0     3.92   168.0   
          3   7.0  245.0     6.34   105.0      2.73   62.0     0.70   127.0   
          4   8.0  162.0     4.19    67.0      1.74   33.0     0.37    88.0   

             LBDLDLSI       Year  
1999-2000 0      3.52  1999-2000  
          1      1.50  1999-2000  
          2      4.34  1999-2000  
          3      3.28  1999-2000  
          4      2.28  1999-2000

In [52]:
sub_names = list(result_1999_2016.columns)
sub_names

['SEQN',
 'LBXTC',
 'LBDTCSI',
 'LBDHDL',
 'LBDHDLSI',
 'LBXTR',
 'LBDTRSI',
 'LBDLDL',
 'LBDLDLSI',
 'Year']

In [36]:
mg_dl = [0,1,3,5,7,9]
sub_names = [sub_names[x] for x in mg_dl]

In [37]:
result_1999_2016_cleaned = result_1999_2016_cleaned[sub_names]

## MongoDB Insertion

In [38]:
#Import MongoClient
from pymongo import MongoClient

In [39]:
#Create a MongoClient to run the MongoDB instance
client = MongoClient("localhost", 27017)

In [40]:
client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [41]:
#Creating a database
db = client['NHANES']
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'NHANES')

In [42]:
db.list_collection_names()

['mcq_h',
 'smq',
 'smqfam',
 'bpx',
 'hiq',
 'demo',
 'bpq',
 'drxtot',
 'huq',
 'tchol',
 'mcq_a',
 'paq',
 'whq',
 'alq',
 'rdq',
 'diq',
 'bmx']

In [43]:
#Creating a collection
tchol = db.tchol

In [44]:
#If collections exist, then drop
if 'tchol' in db.list_collection_names():
    tchol.drop()
    db.list_collection_names()

## Inputting into DB

In [45]:
result_1999_2016_cleaned.rename(columns= {'SEQN':'_id'}, inplace=True)

/Users/changzhoug/anaconda/lib/python3.6/site-packages/pandas/core/frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [46]:
result_1999_2016_cleaned[:3]

_id  LBXTC  LBDHDL  LBXTR  LBDLDL       Year
1999-2000 0  2.0  215.0    54.0  128.0   136.0  1999-2000
          1  3.0  129.0    30.0  202.0    58.0  1999-2000
          2  5.0  279.0    42.0  347.0   168.0  1999-2000

In [47]:
tchol_dict = result_1999_2016_cleaned.to_dict(orient='records')

In [48]:
tchol_dict[0]

{'LBDHDL': 54.0,
 'LBDLDL': 136.0,
 'LBXTC': 215.0,
 'LBXTR': 128.0,
 'Year': '1999-2000',
 '_id': 2.0}

In [49]:
#Insert collection
tchol.insert_many(tchol_dict)

In [50]:
db.list_collection_names()

['mcq_h',
 'smq',
 'smqfam',
 'bpx',
 'hiq',
 'demo',
 'bpq',
 'drxtot',
 'huq',
 'tchol',
 'mcq_a',
 'paq',
 'whq',
 'alq',
 'rdq',
 'diq',
 'bmx']